In [ ]:
from models.yolov5 import YOLOv5
from tracking import ObjectTracker
from visualisation import draw_tracking_result
import cv2

# Define classes
classes = ['person', 'car', 'bus', 'truck']

# Define colors for each class
colors = [[255,0,0], [0,255,0], [0,0,255], [255,255,0]]

# Load YOLOv5 model
model = YOLOv5(weights='yolov5s', classes=classes)

# Define Kalman filter parameters
dt = 1/30.0  # time step
S = np.array([[dt**4/4, 0, dt**3/2, 0],
              [0, dt**4/4, 0, dt**3/2],
              [dt**3/2, 0, dt**2, 0],
              [0, dt**3/2, 0, dt**2]])  # state transition matrix
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])  # measurement matrix
Q = 1e-3 * np.eye(4)  # process noise covariance
R = np.array([[100, 0],
              [0, 100]])  # measurement noise covariance

# Initialize object tracker
tracker = ObjectTracker(classes, colors, S, H, Q, R)

# Load video
cap = cv2.VideoCapture('video.mp4')

# Loop through video frames
while cap.isOpened():
    # Read frame from video
    ret, frame = cap.read()

    # Break if no frames left to read
    if not ret:
        break

    # Perform object detection on the frame
    detections = model.detect(frame)

    # Update object tracker with the detections
    tracker.update(detections)

    # Visualize tracking result
    output_frame = draw_tracking_result(frame, tracker.tracks)

    # Display the resulting frame
    cv2.imshow('Tracking Result', output_frame)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
